<a href="https://colab.research.google.com/github/ekaterga/stc510/blob/main/Module5Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

Here I loaded the dat from the JSON file as a dictionary and converted it to a dataframe

In [2]:
with open('jeopardy.json', 'r') as file:
    data_dict = json.load(file)

In [3]:
data = pd.DataFrame(data_dict)

I wanted to double check that everything looked good. Then I converted the values to strings in order to remove the punctuation in order to convert the values into integers

In [4]:
print(data)

                               category    air_date  \
0                               HISTORY  2004-12-31   
1       ESPN's TOP 10 ALL-TIME ATHLETES  2004-12-31   
2           EVERYBODY TALKS ABOUT IT...  2004-12-31   
3                      THE COMPANY LINE  2004-12-31   
4                   EPITAPHS & TRIBUTES  2004-12-31   
...                                 ...         ...   
216925                   RIDDLE ME THIS  2006-05-11   
216926                        "T" BIRDS  2006-05-11   
216927           AUTHORS IN THEIR YOUTH  2006-05-11   
216928                       QUOTATIONS  2006-05-11   
216929                   HISTORIC NAMES  2006-05-11   

                                                 question  value  \
0       'For the last 8 years of his life, Galileo was...   $200   
1       'No. 2: 1912 Olympian; football star at Carlis...   $200   
2       'The city of Yuma in this state has a record a...   $200   
3       'In 1963, live on "The Art Linkletter Show", t...   $200   

In [5]:
data['value'] = data['value'].astype(str).replace('[\$,]', '', regex=True)
data['value'] = pd.to_numeric(data['value'], errors='coerce').fillna(0).astype(int)

In [6]:
print(data['value'])

0          200
1          200
2          200
3          200
4          200
          ... 
216925    2000
216926    2000
216927    2000
216928    2000
216929       0
Name: value, Length: 216930, dtype: int64


I made any question equal to or more than $800 to high value

In [7]:
data['value_class'] = data['value'].apply(lambda x: 'high' if x >= 800 else 'low')

I split the data into training and testing sets.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data['question'], data['value_class'], test_size=0.2, random_state=42)

Then I vectorized the test data and trained the Naive Bayes classifier

In [9]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

Next I had it make a predicition on the test set, evaluted the classifier, and performed a test question that I had the answer for in the dataset.

In [10]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

MultinomialNB()

In [11]:
y_pred = nb_classifier.predict(X_test_vec)

In [12]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.595514682155534


In [17]:
question = 'This lake lies in a caldera formed when Oregons Mount Mazama volcano collapsed 7,000 years ago'
question_vec = vectorizer.transform([question])
prediction = nb_classifier.predict(question_vec)
print(f'Prediction for "{question}": {prediction}')

Prediction for "This lake lies in a caldera formed when Oregons Mount Mazama volcano collapsed 7,000 years ago": ['high']


The code accurately predicted the value for this question